# quality_assurance

> In progress, including development of diagnostics to be moved elsewhere when complete.

In [ ]:
# |default_exp quality_assurance

In [ ]:
# |export

from abc import ABC, abstractmethod

import dask
import numpy as np
import pandas as pd
import xarray as xr

from qagmire.data import (
    get_lr_l1_single_files,
    get_lr_l2_stack_files,
    read_class_spec,
    read_class_table,
    read_galaxy_table,
    read_primary_header,
)
from qagmire.utilities import parse_obstemp

To write checks of the data, we create a subclass of `Diagnostics` and implement the `tests` method.

In [ ]:
# |export


class Diagnostics(ABC):
    """An abstract class to be subclassed to perform specific diagnostic checks.

    A subclass should perform a set of checks, implemented in a method named `tests`.

    Calling the method `run` will combine and compute the tests, returning the results
    as a single boolean `DataArray` for further analysis.
    """

    def run(self, **kwargs) -> xr.DataArray:
        """Compute the results of the tests.

        The `kwargs` are passed to `qagmire.data.read_*` functions to obtain the data
        for the tests.
        """
        tests = self.tests(**kwargs)
        test_names = [t["name"] for t in tests]
        test_array = [t["test"] for t in tests]
        detail = xr.concat(test_array, pd.Index(test_names, name="test"))
        detail = dask.compute(detail)[0]
        return detail

    @abstractmethod
    def tests(self, **kwargs):
        """Return the tests to be performed.

        Implementations of this method must pass `kwargs` to `qagmire.data.read_*` functions
        as necessary to obtain the data for the tests.

        This method must returns a list of dictionaries with the structure:
        ```
        [
            {
                "name": "a_short_name",
                "description": "The question that the test answers",
                "test": test_dataset,
            },
            ...
        ]
        ```
        where each `test_dataset` should be a boolean `xr.DataArray` of the same shape, giving
        the results of running the test on the data defined by `kwargs`.
        """
        return [
            {
                "name": "a_short_name",
                "description": "The question that the test answers",
                "test": None,
            },
        ]

## Diagnostics

Now let's look at some diagnostic tests.

### Observing conditions check

A reproduction of the weaveio [obs_cond_check](https://github.com/bamford/QAG/blob/master/diagnostics/obs_cond_checks.py).

This tests for the following cases:

* Is the sky brighter than the requirement?
* Is the seeing worse than the requirement?

and also some supplementary tests:

* Are there the other than two runs with the same MJD?
* Do runs with the same MJD have different sky brightness?
* Do runs with the same MJD have different seeing?

In [ ]:
# |export


def _restore_coords(coords, da):
    return [d.assign_coords(coords) for d in da]


def _get_and_check_by_exp(col):
    coords = (
        col.swap_dims(filename="MJD")
        .coords.to_dataset()
        .reset_coords()
        .groupby("MJD")
        .first()
    )
    by_exp = col.groupby("MJD")
    count, first, last = _restore_coords(
        coords, (by_exp.count(), by_exp.first(), by_exp.last())
    )
    expected_runs = count == 2
    runs_match = first == last
    return first, expected_runs, runs_match


def _get_and_check_by_ob(col):
    by_ob = col.groupby("OBID")
    count, first = (by_ob.count(), by_ob.first())
    expected_runs = count == 6
    runs_match = (first != col).any(axis=-1)
    return first, expected_runs, runs_match

In [ ]:
# |export


class ObsCondCheck(Diagnostics):
    def tests(
        self,
        sky_perc_tolerance: float = 0.0,
        seeing_perc_tolerance: float = 0.0,
        **kwargs,
    ):
        files = get_lr_l1_single_files(**kwargs)
        hdr = read_primary_header(files)

        obstemp, two_runs, obstemp_runs_match = _get_and_check_by_ob(hdr["OBSTEMP"])
        obs = parse_obstemp(obstemp)

        sky, _, sky_runs_match = _get_and_check_by_ob(hdr["SKYBRTEL"])
        sky_fail = sky < obs["sky_brightness"] * (1 - sky_perc_tolerance)
        seeing, _, seeing_runs_match = _get_and_check_by_ob(hdr["SEEINGB"])
        seeing_fail = seeing > obs["seeing"] * (1 + seeing_perc_tolerance)

        tests = [
            {
                "name": "sky_too_bright",
                "description": "Is the sky brighter than the requirement?",
                "test": ~sky_fail,
            },
            {
                "name": "seeing_too_bright",
                "description": "Is the seeing worse than the requirement?",
                "test": ~seeing_fail,
            },
            {
                "name": "wrong_run_count",
                "description": "Are there the other than six runs in each OB?",
                "test": ~two_runs,
            },
            {
                "name": "unmatched_runs_sky",
                "description": "Do runs in the same OB have different sky brightness?",
                "test": ~sky_runs_match,
            },
            {
                "name": "unmatched_runs_seeing",
                "description": "Do runs in the same OB have different seeing?",
                "test": ~seeing_runs_match,
            },
        ]
        return tests

In [ ]:
detail = ObsCondCheck().run(date="201*")

Reading files: 100%|██████████| 126/126 [00:09<00:00, 13.48it/s]
Creating Dataset... took 1.93 s. Size is 0.799 Mb


In [ ]:
detail

<xarray.DataArray (test: 5, OBID: 21)>
array([[ True,  True,  True,  True,  True, False,  True,  True,  True,
         True, False,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False]])
Coordinates:
  * OBID     (OBID) int64 3133 3170 3175 3189 3191 ... 3900 3936 4321 4407 4464
  * test     (test) object 'sky_too_bright' ... 'unmatched_runs_seeing'

In [ ]:
per_ob = detail.sum(dim=["OBID"])

In [ ]:
per_ob.to_dataframe("fails")

,fails
test,
sky_too_bright,19
seeing_too_bright,21
wrong_run_count,0
unmatched_runs_sky,0
unmatched_runs_seeing,0


In [ ]:
df = detail.to_dataframe("failed")
df = df.unstack("test").dropna(axis="columns")
df.loc[:, df.any(axis="rows")]

failed                  
test sky_too_bright seeing_too_bright
OBID                                 
3133           True              True
3170           True              True
3175           True              True
3189           True              True
3191           True              True
3217          False              True
3295           True              True
3346           True              True
3372           True              True
3380           True              True
3434          False              True
3653           True              True
3756           True              True
3802           True              True
3803           True              True
3806           True              True
3900           True              True
3936           True              True
4321           True              True
4407           True              True
4464           True              True

### Line flux check

A reproduction of the weaveio [line_flux_check](https://github.com/bamford/QAG/blob/master/diagnostics/line_flux_check.py).

This tests for the following cases:

* Do non-null line fluxes appear in completely null spectra?
* Do non-null line fluxes appear in the blue chip gap?
* Do non-null line fluxes appear in the red chip gap?
* Do non-null line fluxes appear outside the observed wavelength range?
* Do null line fluxes appear in an observed wavelength range?

In [ ]:
# | export


def line_wavelengths(
    galaxy_table: xr.Dataset,  # provides the wavelengths of all lines in the data
    class_table: xr.Dataset,  # provides the redshift of each spectrum
) -> xr.Dataset:  # the observed wavelength of every potential line
    """Determine the expected observed wavelengths of all potential lines."""
    lines = galaxy_table["LINE"].astype(str)
    line_species, line_rest_wl = lines.str.split(dim="", sep="_").T
    line_rest_wl = line_rest_wl.astype(float)
    line_wl = (1 + class_table["Z"]) * line_rest_wl
    return line_wl


def wavelength_boundaries(
    class_spec: xr.Dataset,  # provides the rebinned spectra to check
) -> tuple[dict, dict]:  # the determined boundaries
    """Determine wavelength boundaries and wavelength gaps of blue and red spectra.

    Where a spectrum is entirely null, the returned gaps and boundaries will also be null.

    Returns two dictionaries, `boundaries` and `gaps`, each containing `low` and `high` entries,
    which are Datasets giving the low and high boundaries and gap edges determined for each spectrum.
    """
    gaps = {}
    boundaries = {}
    for band, low, high in (("B", 4000, 6000), ("R", 6000, 9000)):
        wl_dim = f"LAMBDA_{band}"
        wl = class_spec[wl_dim]
        null_flux = class_spec[f"FLUX_RR_{band}"].isnull()
        wl_null = wl.where(null_flux & (wl > low) & (wl < high))
        wl_not_null = wl.where(~null_flux)
        with np.errstate(invalid="ignore"):
            gaps[band] = {
                "low": wl_null.min(dim=wl_dim),
                "high": wl_null.max(dim=wl_dim),
            }
            boundaries[band] = {
                "low": wl_not_null.min(dim=wl_dim),
                "high": wl_not_null.max(dim=wl_dim),
            }
    return boundaries, gaps


def fails_summary(da, top=None):
    df = da.to_dataframe(name="fails").unstack("test")
    df.loc[:, ("fails", "total")] = df.sum(axis="columns")
    df = df.sort_values(("fails", "total"), ascending=False)
    if top is not None:
        df = df.iloc[:top]
    return df

In [ ]:
# |export


class LineFluxCheck(Diagnostics):
    def tests(self, **kwargs):
        lr_l2_stack_files = get_lr_l2_stack_files(**kwargs)

        class_spec = read_class_spec(lr_l2_stack_files)
        galaxy_table = read_galaxy_table(lr_l2_stack_files)
        class_table = read_class_table(lr_l2_stack_files)

        line_wl = line_wavelengths(galaxy_table, class_table)
        boundaries, gaps = wavelength_boundaries(class_spec)

        measured_line_flux = galaxy_table["LINES"].sel(QTY="FLUX", drop=True)
        null_flux = measured_line_flux.isnull()

        is_in_red_gap = (line_wl > gaps["R"]["low"]) & (line_wl < gaps["R"]["high"])
        is_in_blue_gap = (line_wl > gaps["B"]["low"]) & (line_wl < gaps["B"]["high"])

        # ignore gaps in completely null spectra
        is_in_red_gap = is_in_red_gap.fillna(False)
        is_in_blue_gap = is_in_blue_gap.fillna(False)

        is_in_gap = is_in_blue_gap | is_in_red_gap

        is_off_spectrum = (
            (line_wl < boundaries["B"]["low"]) | (line_wl > boundaries["B"]["high"])
        ) & ((line_wl < boundaries["R"]["low"]) | (line_wl > boundaries["R"]["high"]))

        is_on_spectrum = ~is_in_gap & ~is_off_spectrum

        # ignore whether on/off spectrum for completely null spectra
        is_off_spectrum = is_off_spectrum.fillna(False)
        is_on_spectrum = is_in_blue_gap.fillna(False)

        null_spectrum = (
            boundaries["B"]["low"].isnull() | boundaries["R"]["low"].isnull()
        )

        tests = [
            {
                "name": "line_in_null_spectrum",
                "description": "Do non-null line fluxes appear in completely null spectra?",
                "test": ~null_flux & null_spectrum,
            },
            {
                "name": "line_in_blue_chip_gap",
                "description": "Do non-null line fluxes appear in the blue chip gap?",
                "test": ~null_flux & is_in_blue_gap,
            },
            {
                "name": "line_in_red_chip_gap",
                "description": "Do non-null line fluxes appear in the red chip gap?",
                "test": ~null_flux & is_in_red_gap,
            },
            {
                "name": "line_off_spectrum",
                "description": "Do non-null line fluxes appear outside the observed wavelength range?",
                "test": ~null_flux & is_off_spectrum,
            },
            {
                "name": "null_line_on_spectrum",
                "description": "Do null line fluxes appear in an observed wavelength range?",
                "test": null_flux & is_on_spectrum,
            },
        ]
        return tests

In [ ]:
detail = LineFluxCheck().run(date="201*")

Locating and converting where necessary: 100%|██████████| 17/17 [00:00<00:00, 16539.82it/s]
Reading netCDF files... took 1.36 s. Size is 4851.652 Mb
Locating and converting where necessary: 100%|██████████| 17/17 [00:00<00:00, 15343.91it/s]
Reading netCDF files... took 2.17 s. Size is 77.962 Mb
Locating and converting where necessary: 100%|██████████| 17/17 [00:00<00:00, 15271.61it/s]
Reading netCDF files... took 2.70 s. Size is 509.241 Mb


In [ ]:
detail = detail.swap_dims({"filename": "OBID"}).drop_vars("filename")

In [ ]:
per_obid = detail.sum(dim=["APS_ID", "LINE"])
fails_summary(per_obid)

fails                                             \
test line_in_null_spectrum line_in_blue_chip_gap line_in_red_chip_gap   
OBID                                                                    
3900                     0                   162                  226   
3372                     0                   144                  178   
3756                     0                   161                  194   
3653                     0                   165                  196   
3295                     0                   141                  200   
3803                     0                   164                  195   
3806                     0                   165                  202   
3802                     0                   158                  185   
3217                     0                    37                   87   
3346                     0                    22                   94   
3434                     0                    37                   63   
3175                     0                    52                   44   
3380                     0                    32                   46   
3133                     0                    10                   22   
3189                     0                    20                   26   
3170                     0                    15                   37   
3191                     0                     5                    7   

                                                    
test line_off_spectrum null_line_on_spectrum total  
OBID                                                
3900              2646                     2  3036  
3372              2695                     4  3021  
3756              2651                     1  3007  
3653              2558                     1  2920  
3295              2541                     0  2882  
3803              2519                     3  2881  
3806              2497                     4  2868  
3802              2508                     5  2856  
3217              1781                     1  1906  
3346              1684                     1  1801  
3434               747                     0   847  
3175               709                     0   805  
3380               674                     0   752  
3133               288                     1   321  
3189               267                     2   315  
3170               261                     0   313  
3191                97                     0   109

In [ ]:
df = per_obid.to_dataframe(name="fails").unstack("test")
df

fails                                             \
test line_in_null_spectrum line_in_blue_chip_gap line_in_red_chip_gap   
OBID                                                                    
3191                     0                     5                    7   
3133                     0                    10                   22   
3346                     0                    22                   94   
3434                     0                    37                   63   
3372                     0                   144                  178   
3170                     0                    15                   37   
3189                     0                    20                   26   
3217                     0                    37                   87   
3380                     0                    32                   46   
3175                     0                    52                   44   
3295                     0                   141                  200   
3802                     0                   158                  185   
3900                     0                   162                  226   
3803                     0                   164                  195   
3653                     0                   165                  196   
3806                     0                   165                  202   
3756                     0                   161                  194   

                                              
test line_off_spectrum null_line_on_spectrum  
OBID                                          
3191                97                     0  
3133               288                     1  
3346              1684                     1  
3434               747                     0  
3372              2695                     4  
3170               261                     0  
3189               267                     2  
3217              1781                     1  
3380               674                     0  
3175               709                     0  
3295              2541                     0  
3802              2508                     5  
3900              2646                     2  
3803              2519                     3  
3653              2558                     1  
3806              2497                     4  
3756              2651                     1

In [ ]:
per_line = detail.sum(dim=["OBID", "APS_ID"])
fails_summary(per_line)

fails                        \
test            line_in_null_spectrum line_in_blue_chip_gap   
LINE                                                          
[ArIII]_7135.67                     0                     0   
[SII2]_6730.68                      0                     0   
[SII]_6716.31                       0                     0   
[NII]_6583.34                       0                     0   
Ha_6562.80                          0                     0   
[OI]_6300.20                        0                     0   
HeI_5875.60                         0                     1   
HeII_3203.15                        0                   160   
[NeV]_3345.81                       0                   135   
[NeV]_3425.81                       0                   115   
[NI]_5200.39                        0                    51   
[NI]_5197.90                        0                    51   
[OIII]_5006.77                      0                    45   
[NeIII]_3967.40                     0                    81   
[ArIV]_4711.30                      0                    84   
HeII_4685.74                        0                    84   
[OIII]_4363.15                      0                   136   
Hb_4861.32                          0                    44   
[ArIV]_4740.10                      0                    50   
[OII]_3726.03                       0                   178   
[OII]_3728.73                       0                   172   
[NeIII]_3868.69                     0                   103   

                                                                              \
test            line_in_red_chip_gap line_off_spectrum null_line_on_spectrum   
LINE                                                                           
[ArIII]_7135.67                   55              4343                     0   
[SII2]_6730.68                    53              3583                     0   
[SII]_6716.31                     51              3576                     0   
[NII]_6583.34                     47              3361                     0   
Ha_6562.80                        52              3313                     0   
[OI]_6300.20                      73              2776                     0   
HeI_5875.60                      136              1837                     1   
HeII_3203.15                       0              1159                     2   
[NeV]_3345.81                      0               969                     0   
[NeV]_3425.81                      0               871                     1   
[NI]_5200.39                     115               569                     0   
[NI]_5197.90                     118               566                     0   
[OIII]_5006.77                   182               200                     0   
[NeIII]_3967.40                  229                 0                     4   
[ArIV]_4711.30                   211                 0                     1   
HeII_4685.74                     191                 0                     0   
[OIII]_4363.15                   137                 0                     0   
Hb_4861.32                       194                 0                     1   
[ArIV]_4740.10                   158                 0                     1   
[OII]_3726.03                      0                 0                     2   
[OII]_3728.73                      0                 0                     2   
[NeIII]_3868.69                    0                 0                    10   

                       
test            total  
LINE                   
[ArIII]_7135.67  4398  
[SII2]_6730.68   3636  
[SII]_6716.31    3627  
[NII]_6583.34    3408  
Ha_6562.80       3365  
[OI]_6300.20     2849  
HeI_5875.60      1975  
HeII_3203.15     1321  
[NeV]_3345.81    1104  
[NeV]_3425.81     987  
[NI]_5200.39      735  
[NI]_5197.90      735  
[OIII]_5006.77    427  
[NeIII]_3967.40   314  
[ArIV]_4711.30    296  
HeII_4685.74      275  

In [ ]:
per_fibre = detail.sum(dim=["OBID", "LINE"])
fails_summary(per_fibre, top=20)

fails                                             \
test   line_in_null_spectrum line_in_blue_chip_gap line_in_red_chip_gap   
APS_ID                                                                    
456                        0                     0                    5   
766                        0                     1                    7   
728                        0                     3                    3   
989                        0                     2                    8   
746                        0                     1                    1   
40                         0                     1                    5   
615                        0                     5                   15   
308                        0                     4                    3   
62                         0                     2                    6   
273                        0                     2                    3   
310                        0                     0                    6   
905                        0                     2                    7   
491                        0                    13                    6   
700                        0                     3                    2   
579                        0                     3                    0   
280                        0                     6                    3   
469                        0                     8                    8   
738                        0                     0                    4   
282                        0                     2                    9   
764                        0                     7                    1   

                                                      
test   line_off_spectrum null_line_on_spectrum total  
APS_ID                                                
456                   79                     0    84  
766                   74                     0    82  
728                   74                     0    80  
989                   69                     0    79  
746                   72                     0    74  
40                    67                     0    73  
615                   53                     0    73  
308                   65                     0    72  
62                    64                     0    72  
273                   67                     0    72  
310                   65                     0    71  
905                   62                     0    71  
491                   52                     0    71  
700                   64                     0    69  
579                   64                     0    67  
280                   58                     0    67  
469                   50                     0    66  
738                   61                     0    65  
282                   54                     0    65  
764                   56                     0    64

In [ ]:
# |hide
import nbdev

nbdev.nbdev_export()